<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# Chapter 3: Coding Attention Mechanisms

Packages that are being used in this notebook:

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.4.0


- 이 장에서는 LLM의 핵심 동력인 주의 메커니즘에 대해 다룹니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/01.webp?123" width="500px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/02.webp" width="600px">

## 3.1 The problem with modeling long sequences

- No code in this section
- Translating a text word by word isn't feasible due to the differences in grammatical structures between the source and target languages:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/03.webp" width="400px">

- 트랜스포머 모델이 도입되기 전에는 기계 번역 작업에 인코더-디코더 RNN이 일반적으로 사용되었습니다.
- 이 구성에서 인코더는 소스 언어의 토큰 시퀀스를 처리하고, 신경망 내의 중간 계층과 같은 은닉 상태를 사용하여 전체 입력 시퀀스의 압축된 표현을 생성합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/04.webp" width="500px">

## 3.2 Capturing data dependencies with attention mechanisms

- 이 섹션에는 코드가 없습니다.
- 어텐션 메커니즘을 통해 네트워크의 텍스트 생성 디코더 부분은 모든 입력 토큰에 선택적으로 접근할 수 있으며, 이는 특정 입력 토큰이 특정 출력 토큰 생성에 있어 다른 토큰보다 더 중요한 의미를 갖는다는 것을 의미합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/05.webp" width="500px">


- 트랜스포머에서 셀프 어텐션은 시퀀스의 각 위치가 동일 시퀀스 내의 다른 모든 위치와 상호 작용하고 그 관련성을 판단할 수 있도록 함으로써 입력 표현을 향상시키도록 설계된 기술입니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/06.webp" width="300px">

## 3.3 Attending to different parts of the input with self-attention

### 3.3.1 A simple self-attention mechanism without trainable weights

이 섹션에서는 **학습 가능한 가중치(trainable weights)**가 포함되지 않은, 매우 단순화된 형태의 셀프 어텐션 변형을 설명합니다.  
이는 순수하게 설명을 돕기 위한 목적이며, 트랜스포머(Transformers)에서 실제로 사용되는 어텐션 메커니즘이 아닙니다.  
다음 섹션인 3.3.2 섹션에서 이 단순한 어텐션 메커니즘을 확장하여, 실제 셀프 어텐션 메커니즘을 구현할 예정입니다.  
$x^{(1)}$부터 $x^{(T)}$까지의 입력 시퀀스가 주어졌다고 가정해 봅시다.  
입력은 2장에서 설명한 대로 이미 토큰 임베딩으로 변환된 텍스트(예: "Your journey starts with one step"과 같은 문장)입니다.  
예를 들어, $x^{(1)}$은 "Your"라는 단어를 나타내는 $d$차원 벡터이며, 나머지 요소들도 이와 같습니다.  
목표: $x^{(1)}$부터 $x^{(T)}$까지의 각 입력 시퀀스 요소 $x^{(i)}$에 대응하는 **컨텍스트 벡터(context vectors) $z^{(i)}$**를 계산하는 것입니다.  
(여기서 $z$와 $x$는 동일한 차원을 가집니다.)컨텍스트 벡터 $z^{(i)}$는 입력 $x^{(1)}$부터 $x^{(T)}$까지의 **가중합(weighted sum)**입니다.  
컨텍스트 벡터는 특정 입력에 대해 "문맥(context)"-특이적입니다.  
임의의 입력 토큰을 나타내는 자리표시자(placeholder)인 $x^{(i)}$ 대신, 구체적으로 두 번째 입력인 $x^{(2)}$를 생각해 봅시다.또한 구체적인 예를 이어가기 위해, 자리표시자 $z^{(i)}$ 대신 두 번째 출력 컨텍스트 벡터인 $z^{(2)}$를 고려해 봅니다.  
두 번째 컨텍스트 벡터 $z^{(2)}$는 두 번째 입력 요소 $x^{(2)}$를 기준으로, 모든 입력 $x^{(1)}$부터 $x^{(T)}$까지에 가중치를 부여하여 더한 가중합입니다.  
**어텐션 가중치(attention weights)**는 $z^{(2)}$를 계산할 때 각 입력 요소가 가중합에 얼마나 기여할지를 결정하는 값입니다.  
요약하자면, $z^{(2)}$를 **"주어진 작업에 관련된 다른 모든 입력 요소의 정보를 통합하여 수정한 $x^{(2)}$의 버전"**으로 생각하면 됩니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/07.webp" width="400px">

- (이 그림의 숫자는 시각적 혼란을 줄이기 위해 소수점 이하 한 자리까지만 표시되도록 잘렸습니다. 마찬가지로 다른 그림에도 잘린 값이 포함될 수 있습니다.)

- By convention, the unnormalized attention weights are referred to as **"attention scores"** whereas the normalized attention scores, which sum to 1, are referred to as **"attention weights"**



- 아래 코드는 위 그림을 단계별로 따라갑니다.
<br>

- **1단계:** 정규화되지 않은 주의력 점수 $\omega$를 계산합니다.
- 두 번째 입력 토큰을 쿼리로 사용한다고 가정하면, 즉 $q^{(2)} = x^{(2)}$인 경우, 내적을 통해 정규화되지 않은 어텐션 점수를 계산합니다.
    - $\omega_{21} = x^{(1)} q^{(2)\top}$
    - $\omega_{22} = x^{(2)} q^{(2)\top}$
    - $\omega_{23} = x^{(3)} q^{(2)\top}$
    - ...
    - $\omega_{2T} = x^{(T)} q^{(2)\top}$

- 위에서 $\omega$는 정규화되지 않은 주의력 점수를 나타내는 데 사용되는 그리스 문자 "오메가"입니다.

- $\omega_{21}$의 아래첨자 "21"은 입력 시퀀스 요소 2가 입력 시퀀스 요소 1에 대한 쿼리로 사용되었음을 의미합니다.

- 3장에서 설명한 대로 3차원 벡터에 이미 임베딩된 다음 입력 문장이 있다고 가정해 보겠습니다(설명을 위해 줄 바꿈 없이 페이지에 맞도록 매우 작은 임베딩 차원을 사용합니다).

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

- (본 책에서는 머신러닝 및 딥러닝에서 흔히 사용되는 관례에 따라 훈련 예제를 행으로, 특징 값을 열로 표현합니다. 위 그림의 텐서에서 각 행은 단어를 나타내고, 각 열은 임베딩 차원을 나타냅니다.)

- 이 섹션의 주요 목적은 두 번째 입력 시퀀스 $x^{(2)}$를 쿼리로 사용하여 컨텍스트 벡터 $z^{(2)}$를 계산하는 방법을 보여주는 것입니다.

- 그림은 이 과정의 첫 번째 단계를 나타냅니다. 이 단계에서는 내적 연산을 통해 $x^{(2)}$와 다른 모든 입력 요소 간의 어텐션 점수 ω를 계산합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="400px">

- 입력 시퀀스 요소 2인 $x^{(2)}$를 예시로 사용하여 컨텍스트 벡터 $z^{(2)}$를 계산합니다. 이 섹션 후반부에서 이를 일반화하여 모든 컨텍스트 벡터를 계산할 것입니다.
- 첫 번째 단계는 쿼리 $x^{(2)}$와 다른 모든 입력 토큰 간의 내적을 계산하여 정규화되지 않은 어텐션 점수를 계산하는 것입니다.

In [3]:
query = inputs[1]  # 2nd input token is the query

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- 참고: 내적은 기본적으로 두 벡터의 요소를 곱하고 그 결과를 모두 더하는 것을 간략하게 표현한 것입니다.

In [4]:
res = 0.

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


- **2단계:** 정규화되지 않은 어텐션 점수("오메가", $\omega$)의 합이 1이 되도록 정규화합니다.
- 정규화되지 않은 어텐션 점수의 합을 1로 만드는 간단한 방법은 다음과 같습니다(해석에 유용하고 훈련 안정성에 중요한 관례입니다).

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/09.webp" width="500px">

In [5]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)



- 하지만 실제로는 극단값을 더 잘 처리하고 학습 중 기울기 특성이 더 바람직한 소프트맥스 함수를 정규화에 사용하는 것이 일반적이며 권장됩니다.
- 다음은 벡터 요소의 합이 1이 되도록 정규화하는 동시에 스케일링을 위한 소프트맥스 함수의 간단한 구현입니다.

In [6]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- 위와 같은 단순한 구현 방식은 오버플로우 및 언더플로우 문제로 인해 입력값이 크거나 작을 경우 수치적 불안정성을 보일 수 있습니다.
- 따라서 실제로는 성능 최적화가 잘 되어 있는 PyTorch의 소프트맥스 구현을 사용하는 것이 좋습니다.

In [7]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- **3단계**: 내장된 입력 토큰 $x^{(i)}$와 어텐션 가중치를 곱하여 컨텍스트 벡터 $z^{(2)}$를 계산하고 결과 벡터들을 합산합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/10.webp" width="500px">

In [8]:
query = inputs[1] # 2nd input token is the query

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 Computing attention weights for all input tokens

#### 모든 입력 시퀀스 토큰으로 일반화:

- 위에서 입력 2에 대한 어텐션 가중치와 컨텍스트 벡터를 계산했습니다(아래 그림의 강조 표시된 행 참조).
- 다음으로, 이 계산을 일반화하여 모든 어텐션 가중치와 컨텍스트 벡터를 계산합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/11.webp" width="400px">

- (이 그림의 숫자는 시각적 혼란을 줄이기 위해 소수점 이하 두 자리까지만 표시되도록 잘렸습니다. 각 행의 값들을 더하면 1.0 또는 100%가 됩니다. 마찬가지로 다른 그림의 숫자도 잘렸습니다.)

- 셀프 어텐션에서는 먼저 어텐션 점수를 계산하고, 이를 정규화하여 합계가 1이 되는 어텐션 가중치를 도출합니다.
- 이 어텐션 가중치는 입력값들의 가중 합산을 통해 컨텍스트 벡터를 생성하는 데 사용됩니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/12.webp" width="400px">


- 이전 **1단계**를 모든 쌍 요소에 적용하여 정규화되지 않은 어텐션 점수 행렬을 계산합니다.

In [9]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 위와 동일한 결과를 행렬 곱셈을 통해 더욱 효율적으로 얻을 수 있습니다.

In [10]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 이전의 **2단계**와 유사하게, 각 행의 값들의 합이 1이 되도록 각 행을 정규화합니다.

In [11]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- 각 행의 값들의 합이 실제로 1인지 빠르게 확인합니다.

In [12]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)

print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- 이전 **3단계**를 적용하여 모든 컨텍스트 벡터를 계산합니다.

In [13]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


- 정상 작동 여부를 확인하기 위해 이전에 계산된 컨텍스트 벡터 $z^{(2)} = [0.4419, 0.6515, 0.5683]$는 위의 2번째 행에서 찾을 수 있습니다.

In [14]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## 3.4 Implementing self-attention with trainable weights

- 이 절에서 개발된 자기 주의 메커니즘이 이 책과 이 장의 전체적인 서술 및 구조에 어떻게 통합되는지를 보여주는 개념적 틀

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/13.webp" width="400px">

### 3.4.1 Computing the attention weights step by step

- 이 섹션에서는 오리지널 트랜스포머 아키텍처, GPT 모델 및 대부분의 인기 있는 LLM에서 사용되는 셀프 어텐션 메커니즘을 구현합니다.
- 이 셀프 어텐션 메커니즘은 "스케일링된 도트 프로덕트 어텐션"이라고도 합니다.
- 전반적인 아이디어는 이전과 유사합니다.
- 특정 입력 요소에 대한 입력 벡터의 가중 합으로 컨텍스트 벡터를 계산하고자 합니다.
- 이를 위해서는 어텐션 가중치가 필요합니다.
- 보시다시피, 이전에 소개된 기본 어텐션 메커니즘과 비교했을 때 약간의 차이점만 있습니다.

- 가장 눈에 띄는 차이점은 모델 학습 중에 업데이트되는 가중치 행렬의 도입입니다.

- 이러한 학습 가능한 가중치 행렬은 모델(특히 모델 내부의 어텐션 모듈)이 "좋은" 컨텍스트 벡터를 생성하도록 학습하는 데 매우 중요합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/14.webp" width="600px">

- 셀프 어텐션 메커니즘을 단계별로 구현하기 위해, 먼저 세 가지 학습 가중치 행렬 $W_q$, $W_k$, $W_v$를 소개합니다.
- 이 세 행렬은 임베딩된 입력 토큰 $x^{(i)}$를 행렬 곱셈을 통해 쿼리, 키, 값 벡터로 변환하는 데 사용됩니다.

- 쿼리 벡터: $q^{(i)} = x^{(i)}\,W_q$

- 키 벡터: $k^{(i)} = x^{(i)}\,W_k$

- 값 벡터: $v^{(i)} = x^{(i)}\,W_v$

- 입력 벡터 $x$와 쿼리 벡터 $q$의 임베딩 차원은 모델 설계 및 구체적인 구현 방식에 따라 같을 수도 있고 다를 수도 있습니다.
- GPT 모델에서는 일반적으로 입력과 출력 차원이 같지만, 계산 과정을 더 잘 이해하기 위해 여기서는 입력과 출력 차원을 다르게 설정했습니다.

In [15]:
x_2 = inputs[1] # second input element
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2

- 아래에서는 세 개의 가중치 행렬을 초기화합니다. 설명 목적으로 출력 화면을 간소화하기 위해 `requires_grad=False`로 설정했지만, 모델 학습에 가중치 행렬을 사용하려면 학습 중에 행렬이 업데이트되도록 `requires_grad=True`로 설정해야 합니다.

In [16]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

- 다음으로 쿼리, 키 및 값 벡터를 계산합니다.

In [17]:
query_2 = x_2 @ W_query # _2 because it's with respect to the 2nd input element
key_2 = x_2 @ W_key 
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


- 아래에서 볼 수 있듯이, 6개의 입력 토큰을 3차원 공간에서 2차원 임베딩 공간으로 성공적으로 투영했습니다.

In [18]:
keys = inputs @ W_key 
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


- 다음 단계인 **2단계**에서는 쿼리 벡터와 각 키 벡터의 내적을 계산하여 정규화되지 않은 어텐션 점수를 계산합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/15.webp" width="600px">

In [19]:
keys_2 = keys[1] # Python starts index at 0
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


- 입력값이 6개이므로, 주어진 쿼리 벡터에 대해 6개의 어텐션 스코어가 있습니다.


In [20]:
attn_scores_2 = query_2 @ keys.T # All attention scores for given query
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/16.webp" width="600px">

- 다음으로, **3단계**에서는 이전에 사용했던 소프트맥스 함수를 이용하여 어텐션 가중치(합이 1이 되는 정규화된 어텐션 점수)를 계산합니다.
- 이전과 다른 점은 이제 어텐션 점수를 임베딩 차원의 제곱근, 즉 $\sqrt{d_k}$(`d_k**0.5`)로 나누어 스케일링한다는 것입니다.

In [21]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/17.webp" width="600px">


- **4단계**에서는 입력 쿼리 벡터 2에 대한 컨텍스트 벡터를 계산합니다.

In [22]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


### 3.4.2 Implementing a compact SelfAttention class


- 이 모든 것을 종합하면, 다음과 같이 셀프 어텐션 메커니즘을 구현할 수 있습니다.

In [23]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        
        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="400px">

- PyTorch의 Linear 레이어를 사용하면 위의 구현을 간소화할 수 있습니다. 바이어스 유닛을 비활성화하면 Linear 레이어는 행렬 곱셈과 동일한 기능을 합니다.
- 수동으로 `nn.Parameter(torch.rand(...))`를 사용하는 방식 대신 `nn.Linear`를 사용하는 또 다른 큰 장점은 `nn.Linear`가 선호하는 가중치 초기화 방식을 제공하여 모델 학습을 더욱 안정적으로 만들어준다는 것입니다.

In [24]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


- `SelfAttention_v1`과 `SelfAttention_v2`는 가중치 행렬에 서로 다른 초기 가중치를 사용하기 때문에 서로 다른 출력을 생성합니다.

## 3.5 Hiding future words with causal attention

- 인과적 어텐션에서는 대각선 위의 어텐션 가중치가 마스킹되어, 주어진 입력에 대해 LLM이 어텐션 가중치를 사용하여 컨텍스트 벡터를 계산할 때 미래의 토큰을 활용할 수 없도록 합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/19.webp" width="400px">

### 3.5.1 Applying a causal attention mask

- 이 섹션에서는 기존의 셀프 어텐션 메커니즘을 인과적 셀프 어텐션 메커니즘으로 변환합니다.
- 인과적 셀프 어텐션은 시퀀스의 특정 위치에 대한 모델의 예측이 이전 위치의 알려진 출력에만 의존하고, 미래 위치에는 의존하지 않도록 합니다.
- 간단히 말하면, 각 다음 단어 예측은 이전 단어에만 의존해야 합니다.
- 이를 위해 각 토큰에 대해 미래 토큰(입력 텍스트에서 현재 토큰 다음에 오는 토큰)을 마스킹합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/20.webp" width="600px">

- 인과적 자기 주의를 설명하고 구현하기 위해 이전 섹션에서 다룬 주의 점수와 가중치를 활용해 보겠습니다.

In [25]:
# Reuse the query and key weight matrices of the
# SelfAttention_v2 object from the previous section for convenience
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs) 
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


- 향후 어텐션 가중치를 마스킹하는 가장 간단한 방법은 PyTorch의 tril 함수를 사용하여 마스크를 생성하는 것입니다. 이 마스크는 주 대각선 아래 요소(대각선 자체 포함)를 1로, 주 대각선 위 요소를 0으로 설정합니다.

In [26]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- 그런 다음, 이 마스크를 어텐션 가중치에 곱하여 대각선 위의 어텐션 점수를 0으로 만들 수 있습니다.

In [27]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


- 하지만 위 예시처럼 소프트맥스 후에 마스크를 적용하면 소프트맥스가 생성한 확률 분포가 왜곡될 수 있습니다.
- 소프트맥스는 모든 출력값의 합이 1이 되도록 합니다.
- 소프트맥스 후에 마스킹을 적용하려면 출력값의 합이 다시 1이 되도록 재정규화해야 하므로 과정이 복잡해지고 의도치 않은 결과가 발생할 수 있습니다.

- 행들의 합이 1이 되도록 하려면 다음과 같이 어텐션 가중치를 정규화할 수 있습니다.

In [28]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


- 이제 인과적 어텐션 메커니즘 코딩은 사실상 완료되었지만, 위와 동일한 결과를 얻는 보다 효율적인 접근 방식을 간단히 살펴보겠습니다.
- 대각선 위의 어텐션 가중치를 0으로 만들고 결과를 정규화하는 대신, 소프트맥스 함수에 들어가기 전에 대각선 위의 정규화되지 않은 어텐션 점수를 음의 무한대로 마스킹할 수 있습니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/21.webp" width="450px">

In [29]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


- 아래에서 볼 수 있듯이, 이제 각 행의 어텐션 가중치 합이 다시 1이 됩니다.

In [30]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 3.5.2 Masking additional attention weights with dropout

- 또한, 학습 중 과적합을 줄이기 위해 드롭아웃을 적용합니다.
- 드롭아웃은 여러 위치에 적용할 수 있습니다.

- 예를 들어, 어텐션 가중치를 계산한 후;

- 또는 어텐션 가중치와 값 벡터를 곱한 후;

- 여기서는 더 일반적인 방법인 어텐션 가중치 계산 후 드롭아웃 마스크를 적용합니다.

- 또한, 이 특정 예제에서는 드롭아웃 비율을 50%로 설정했는데, 이는 어텐션 가중치의 절반을 무작위로 제거하는 것을 의미합니다. (나중에 GPT 모델을 학습할 때는 0.1 또는 0.2와 같은 더 낮은 드롭아웃 비율을 사용할 것입니다.)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/22.webp" width="400px">

- 만약 탈락률을 0.5(50%)로 적용하면, 탈락하지 않은 값은 1/0.5 = 2의 비율로 조정됩니다.
- 조정 비율은 1 / (1 - `탈락률`) 공식으로 계산됩니다.

In [31]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # dropout rate of 50%
example = torch.ones(6, 6) # create a matrix of ones

print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [32]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)


- 드롭아웃 출력 결과는 운영 체제에 따라 다르게 보일 수 있습니다. 이러한 불일치에 대한 자세한 내용은 [PyTorch 이슈 트래커](https://github.com/pytorch/pytorch/issues/121595)에서 확인할 수 있습니다.

### 3.5.3 Implementing a compact causal self-attention class

- 이제 인과 마스크와 드롭아웃 마스크를 포함한 셀프 어텐션의 실제 구현을 진행할 준비가 되었습니다.
- 마지막으로, 2장에서 구현한 데이터 로더에서 생성된 배치 출력을 `CausalAttention` 클래스가 지원할 수 있도록 여러 개의 입력으로 구성된 배치를 처리하는 코드를 구현해야 합니다.
- 간단하게 배치 입력을 시뮬레이션하기 위해 입력 텍스트 예제를 복제합니다.

In [33]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) # 2 inputs with 6 tokens each, and each token has embedding dimension 3

torch.Size([2, 6, 3])


In [34]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        # For inputs where `num_tokens` exceeds `context_length`, this will result in errors
        # in the mask creation further below.
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method. 
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 드롭아웃은 학습 과정에서만 적용되고 추론 과정에서는 적용되지 않는다는 점에 유의하십시오.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/23.webp" width="500px">

## 3.6 Extending single-head attention to multi-head attention

### 3.6.1 Stacking multiple single-head attention layers

- 아래는 이전에 구현한 셀프 어텐션에 대한 요약입니다(간단히 설명하기 위해 인과 관계 및 드롭아웃 마스크는 생략했습니다).

- 이를 싱글 헤드 어텐션이라고도 합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/24.webp" width="400px">

- 단일 헤드 어텐션 모듈 여러 개를 쌓아서 멀티 헤드 어텐션 모듈을 만드는 것입니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/25.webp" width="400px">

- 멀티헤드 어텐션의 핵심 아이디어는 서로 다른 학습된 선형 투영을 사용하여 어텐션 메커니즘을 여러 번(병렬로) 실행하는 것입니다. 이를 통해 모델은 서로 다른 위치에 있는 다양한 표현 부분 공간의 정보에 동시에 주의를 기울일 수 있습니다.

In [35]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- 위 구현에서 임베딩 차원은 4입니다. 키, 쿼리, 값 벡터와 컨텍스트 벡터 모두에 대해 `d_out=2`로 임베딩 차원을 설정했기 때문입니다. 또한 어텐션 헤드가 2개이므로 출력 임베딩 차원은 2*2=4가 됩니다.

### 3.6.2 Implementing multi-head attention with weight splits

- 위 코드는 직관적이고 완벽하게 작동하는 멀티 헤드 어텐션 구현(이전의 싱글 헤드 어텐션 `CausalAttention` 구현을 래핑한 것)이지만, 동일한 기능을 구현하는 `MultiHeadAttention`이라는 독립형 클래스를 작성할 수도 있습니다.

- 이 독립형 `MultiHeadAttention` 클래스에서는 여러 개의 어텐션 헤드를 연결하지 않습니다.
- 대신, W_query, W_key, W_value 가중치 행렬을 각각 생성한 다음, 각 어텐션 헤드에 대해 개별 행렬로 분할합니다.

In [36]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method.

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 위 코드는 `MultiHeadAttentionWrapper`를 더욱 효율적으로 재작성한 버전입니다.
- 무작위 가중치 초기화 방식이 다르기 때문에 출력 결과가 약간 다르게 보일 수 있지만, 두 코드 모두 향후 장에서 구현할 GPT 클래스에서 사용할 수 있는 완전한 기능을 갖춘 구현입니다.

---

**출력 차원에 대한 참고 사항**

- 위의 `MultiHeadAttention`에서 `d_out=2`를 사용한 것은 앞서 설명한 `MultiHeadAttentionWrapper` 클래스와 동일한 설정을 사용하기 위함입니다.
- `MultiHeadAttentionWrapper`는 헤드들을 연결하는 방식으로 인해 출력 헤드 차원을 `d_out * num_heads` (즉, `2*2 = 4`)로 반환합니다.
- 하지만 `MultiHeadAttention` 클래스는 (사용자 편의성을 높이기 위해) `d_out`을 통해 출력 헤드 차원을 직접 제어할 수 있도록 합니다. 즉, `d_out = 2`로 설정하면 헤드 개수와 관계없이 출력 헤드 차원이 2가 됩니다.
- 돌이켜보면, 독자들이 지적했듯이(https://github.com/rasbt/LLMs-from-scratch/pull/859), `MultiHeadAttention`을 `d_out = 4`로 사용하는 것이 `MultiHeadAttentionWrapper`를 `d_out = 2`로 사용할 때와 동일한 출력 차원을 생성하므로 더 직관적일 수 있습니다.

---

- 참고로, 위의 `MultiHeadAttention` 클래스에 선형 투영 레이어(`self.out_proj`)를 추가했습니다. 이는 차원을 변경하지 않는 단순한 선형 변환입니다. LLM 구현에서 이러한 투영 레이어를 사용하는 것은 일반적인 관례이지만, 반드시 필요한 것은 아닙니다(최근 연구에 따르면 모델링 성능에 영향을 미치지 않고 제거할 수 있습니다. 이 장 끝부분의 참고 문헌 섹션을 참조하십시오).

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="400px">

- 참고로, 위 내용을 간결하고 효율적으로 구현하고 싶다면 PyTorch의 `torch.nn.MultiheadAttention` 클래스를 사용하는 것도 고려해 볼 수 있습니다(https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html).

- 위 구현 방식이 처음 보면 다소 복잡해 보일 수 있으므로, `attn_scores = queries @ keys.transpose(2, 3)`를 실행했을 때 어떤 일이 발생하는지 살펴보겠습니다.

In [37]:
# (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

print(a @ a.transpose(2, 3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


- In this case, the matrix multiplication implementation in PyTorch will handle the 4-dimensional input tensor so that the matrix multiplication is carried out between the 2 last dimensions (num_tokens, head_dim) and then repeated for the individual heads 

- For instance, the following becomes a more compact way to compute the matrix multiplication for each head separately:

In [38]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


# Summary and takeaways

- See the [./multihead-attention.ipynb](./multihead-attention.ipynb) code notebook, which is a concise version of the data loader (chapter 2) plus the multi-head attention class that we implemented in this chapter and will need for training the GPT model in upcoming chapters
- You can find the exercise solutions in [./exercise-solutions.ipynb](./exercise-solutions.ipynb)